# Feature selection


In [ ]:
import pandas as pd

data = pd.read_excel('df2.xlsx')

data.to_csv('df2.csv', index=False)


In [ ]:
print (data.dtypes)


In [ ]:
#data = pd.read_csv('FullData.csv')
print(data.columns)
data2=data.copy()


In [ ]:
cat_cols = data.columns
for col in cat_cols:
    data[col] = data[col].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
print(data2)


In [ ]:
from datetime import datetime, timedelta

data['time'] = data.apply(lambda row: (datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%H:%M %p") + pd.Timedelta(hours=12)).strftime("%H:%M:%S") if row['AMPM'] == 'PM' else datetime.strptime(f"{row['TIMEHR']}:{row['TIMEMIN']} {row['AMPM']}", "%I:%M %p").strftime("%H:%M:%S"), axis=1)

data = data.drop(['TIMEHR', 'TIMEMIN', 'AMPM'], axis=1)

print(data)


In [ ]:
print(data[['time']].head())
print(data2[['TIMEHR','TIMEMIN','AMPM']].head())


In [ ]:
print(data.dtypes)


In [ ]:
data['time'] = data['time'].astype(str)
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')

data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

data = data.drop('time', axis=1)

print(data)


In [ ]:
print(data.columns)


In [ ]:
databeforeNA= data.copy()

num_rows = databeforeNA.shape[0]
print("# of records before cleaning", num_rows)
databeforeNA = databeforeNA.dropna()
num_rows = databeforeNA.shape[0]
print("# of records after NA removal", num_rows)


In [ ]:
datawithTime=databeforeNA.copy()
print(datawithTime.head())


In [ ]:
unique_values0 = datawithTime['RAILROAD'].unique()
unique_values1 = datawithTime['STATE '].unique()
unique_values2 = datawithTime['CAUSE'].unique()
unique_values4 = datawithTime['TYPEQ'].unique()
unique_values5 = datawithTime['TRKCLAS'].unique()

print(len(unique_values0))
print(len(unique_values1))
print(len(unique_values2))
print(len(unique_values4))
print(len(unique_values5))


In [ ]:
print(data.dtypes)


In [ ]:
from sklearn.preprocessing import LabelEncoder
datawithTime = databeforeNA.copy()
print(datawithTime['TYPEQ'].unique())
le = LabelEncoder()
cat_cols = ['RAILROAD','STATE ', 'CAUSE']

for col in cat_cols:
    if datawithTime[col].dtype == 'object':
        datawithTime[col] = le.fit_transform(datawithTime[col])

typeq_dict = {'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'a': 10, 'b': 11, 'c': 12, 'd': 13, 'e': 14}
datawithTime['TYPEQ'] = datawithTime['TYPEQ'].astype(str).map(typeq_dict)

print(datawithTime['TYPEQ'].unique())


In [ ]:
print(datawithTime.dtypes)


In [ ]:
print(datawithTime[['STATE ']].head())
print(datawithTime[['RAILROAD']].head())
print(datawithTime[['CAUSE']].head())
print(datawithTime[['TYPEQ']].head())


In [ ]:
print(datawithTime.dtypes)


In [ ]:
print(datawithTime.columns)


In [ ]:

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR','MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

X = data_FSelection[feature_cols]

k = 5

for target_col in target_cols:
    y = data_FSelection[target_col]

    selector = SelectKBest(score_func=f_classif, k=k)
    selector.fit(X, y)

    top_k_indices = selector.get_support(indices=True)

    top_k_features = [feature_cols[i] for i in top_k_indices]
    print('Selected Features:', top_k_features)

    X_top_k = X[top_k_features]
    linreg = LinearRegression()
    scores = cross_val_score(linreg, X_top_k, y, cv=5)

    print(f'Accuracy score for {target_col}: {scores.mean()}')



In [ ]:

import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

k = 5

for target_col in target_cols:
    X = data_FSelection[feature_cols]
    y = data_FSelection[target_col]

    dtr = DecisionTreeRegressor(random_state=42)
    selector = SelectFromModel(dtr, threshold=-np.inf, max_features=k)
    selector.fit(X, y)

    top_k_indices = selector.get_support(indices=True)

    top_k_features = [feature_cols[i] for i in top_k_indices]
    print(f'Selected Features for {target_col}:', top_k_features)

    X_top_k = X[top_k_features]
    dtr = DecisionTreeRegressor(random_state=42)
    scores = cross_val_score(dtr, X_top_k, y, cv=5)

    print(f'R2 score for {target_col}: {np.mean(scores)}')



In [ ]:

import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

k = 5

cv = KFold(n_splits=5, shuffle=True, random_state=42)

for target_col in target_cols:
    X = data_FSelection[feature_cols]
    y = data_FSelection[target_col]

    dtr = DecisionTreeRegressor(random_state=42)
    selector = SelectFromModel(dtr, threshold=-np.inf, max_features=k)
    selector.fit(X, y)

    top_k_indices = selector.get_support(indices=True)

    top_k_features = [feature_cols[i] for i in top_k_indices]
    print(f'Selected Features for {target_col}:', top_k_features)

    X_top_k = X[top_k_features]
    dtr = DecisionTreeRegressor(random_state=42)
    scores = cross_val_score(dtr, X_top_k, y, cv=cv)

    print(f'R2 score for {target_col}: {np.mean(scores)}')



In [ ]:

import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv')

k = 5

cv = KFold(n_splits=5, shuffle=True, random_state=42)

for target_col in target_cols:
    X = data_FSelection[feature_cols]
    y = data_FSelection[target_col]

    svr = SVR(kernel='linear')
    selector = SelectFromModel(svr, threshold=-np.inf, max_features=k)
    selector.fit(X, y)

    top_k_indices = selector.get_support(indices=True)

    top_k_features = [feature_cols[i] for i in top_k_indices]
    print(f'Selected Features for {target_col}:', top_k_features)

    X_top_k = X[top_k_features]
    svr = SVR(kernel='linear')
    scores = cross_val_score(svr, X_top_k, y, cv=cv)

    print(f'R2 score for {target_col}: {np.mean(scores)}')



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv('datawithTime.csv', nrows=int(1*len(pd.read_csv('datawithTime.csv'))))

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

X = data[feature_cols]
y = data[target_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_models = {}
for target in target_cols:
    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train[target])
    rf_models[target] = rf

y_pred = {}
for target in target_cols:
    y_pred[target] = rf_models[target].predict(X_test)

accuracy = {}
for target in target_cols:
    accuracy[target] = accuracy_score(y_test[target], y_pred[target])
    print("Accuracy for", target, ":", accuracy[target])


In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

data = pd.read_csv('datawithTime.csv', nrows=int(1*len(pd.read_csv('datawithTime.csv'))))

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
all_feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                    'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                    'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                    'HIGHSPD', 'hour', 'minute']

X = data[all_feature_cols]
y = data[target_cols]

for target in target_cols:
    y_single = y[target]

    k = 10
    selector = SelectKBest(f_classif, k=k)
    X_new = selector.fit_transform(X, y_single)

    selected_feature_indices = selector.get_support(indices=True)
    selected_feature_cols = [all_feature_cols[i] for i in selected_feature_indices]

    X_train, X_test, y_train, y_test = train_test_split(X_new, y_single, test_size=0.3, random_state=42)

    rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf.fit(X_train, y_train)

    X_test_new = selector.transform(X_test)
    y_pred = rf.predict(X_test_new)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy for", target, ":", accuracy)

    print("Selected feature columns:", selected_feature_cols)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
df = pd.read_csv('datawithTime.csv')

num_rows = len(df)

first_half = df.head(int(num_rows/1000))

X = first_half[feature_cols]
y = first_half[target_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train, y_train)

selector = SelectFromModel(rf, threshold='median')
selector.fit(X_train, y_train)

for i, target in enumerate(target_cols):
    print(f"Selected features for target '{target}':")
    print(X.columns[selector.get_support()][i*5:i*5+5])
    print()

X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

rf_selected = RandomForestRegressor(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

for i, target in enumerate(target_cols):
    r2_score = rf_selected.score(X_test_selected[:, i*5:i*5+5], y_test[target])
    print(f"R^2 score for target '{target}': {r2_score}")


In [ ]:

import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

target_cols = ['ACCDMG','YEAR', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'YEAR', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK',
                'HIGHSPD', 'hour', 'minute']

datawithTime.to_csv('datawithTime.csv', index=False)
data_FSelection = pd.read_csv('datawithTime.csv').sample(frac=0.5, random_state=42)

X = data_FSelection[feature_cols]
y = data_FSelection[target_cols]

k = 5

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
selector = SelectFromModel(rfc, threshold=-np.inf, max_features=k)
selector.fit(X, y)

top_k_indices = selector.get_support(indices=True)

top_k_features = [feature_cols[i] for i in top_k_indices]

X_top_k = X[top_k_features]
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
scores = cross_val_score(rfc, X_top_k, y, cv=5)

for i, target_col in enumerate(target_cols):
    print(f'Accuracy score for {target_col}: {scores[i]}')


In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.23.2
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import cfs as CFS
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

dataFS1 = datawithTime

feature_cols = ['RAILROAD', 'RR3', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK', 'ACCTRKCL',
                'HIGHSPD', 'hour', 'minute']
target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']

for i, target_col in enumerate(target_cols):
    print(f"Processing target variable {i+1} ({target_col})")

    X = dataFS1[feature_cols]
    y = dataFS1[target_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    selected_feature_indexes = CFS(X_train.values, y_train.values)

    features_selected = [feature_cols[i] for i in selected_feature_indexes]

    print(f"Selected features: {features_selected}")



In [ ]:
!pip install numpy==1.16.0
!pip install scikit-learn==0.24

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

df =  datawithTime

target_cols = ['ACCDMG', 'CASKLD', 'CASINJ', 'EVACUATE', 'CARSDMG', 'CARSHZD']
feature_cols = ['RAILROAD', 'RR3', 'MONTH', 'DAY', 'CARS', 'STATE ', 'TEMP', 'VISIBLTY', 'WEATHER',
                'TRNSPD', 'TONS', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'POSITON1',
                'HEADEND1', 'LOADF1', 'EMPTYF1', 'CAUSE', 'ACCTRK', 'ACCTRKCL',
                'HIGHSPD', 'hour', 'minute']
X = df[feature_cols]
y = df[target_cols]

clf = RandomForestClassifier(n_estimators=100)

cv = StratifiedKFold(n_splits=5)

for i, target_col in enumerate(target_cols):
    print(f"Processing target variable {i+1} ({target_col})")
    rfecv = RFECV(estimator=clf, step=1, cv=cv, scoring='roc_auc')
    rfecv.fit(X, y[target_col])

    print(f"Optimal number of features: {rfecv.n_features_}")
    print(f"Selected features: {X.columns[rfecv.support_].tolist()}")
    print(f"Feature ranking: {rfecv.ranking_}")


In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100)

rfecv = RFECV(estimator=rfc, step=1, cv=5, scoring='accuracy')
rfecv.fit(X_train, y_train)

print(f"Optimal number of features: {rfecv.n_features_}")

print("Most important features:")
for i in range(len(rfecv.support_)):
    if rfecv.support_[i]:
        print(f"\t{i}")

